In [1]:
import numpy as np

In [20]:
'''计算单元'''
#电阻计算模块
def R1_calculate(p,s,n=1):
    '''电阻率p,导线横截面积分S，分裂数默认为1  (注意分裂导线S需要乘以n)'''
    #s单位是mm2,p单位是mm2*oumu/km
    return p/(s*n)
#电感与电纳计算计算模块
def X_caculate(dh,r,di,shape=0,dv=0,n=1):
    '''dh:sh水平间距，di:相内间距,shape:,dv:垂直间距，r=计算半径，n：分裂数'''
    #shape可选：正三角形，水平，正四边形，正五边形，正六边形
    #dh与di，dv的单位要统一哦
    d1=np.sqrt(dh**2+dv**2)
    if dv>0:
        deq=np.power(2*dh*d1*d1,1/3)
    else:
        deq=np.power(2*d1*d1*d1,1/3)
    alpha=1
    if n==2:
        alpha=1
    if (n==3) and (shape=='水平'):
        alpha=1.26
    if (n==3) and (shape=='正三角形'):
        alpha=1
    if n==4:
        alpha=1.12
    if n==5:
        alpha=1.27
    if n==6:
        alpha=1.4
    req=np.power(r*np.power(alpha*di,n-1),1/n)
    return 0.1445*np.log10(deq/req)+0.0157/n,7.58e-6/(np.log10(deq/req))

def modify_approximate(x1,b1,r1,l):
    kr=1-(1/3)*x1*b1*l*l
    kx=1-(1/6)*(x1*b1-r1*r1*b1/x1)*l*l
    kb=1+(1/12)*x1*b1*l*l
    return kr,kx,kb



def main_judge(l,p,s,dh,r,di,shape,dv=0,n=1):
    x1,b1=X_caculate(dh,r,di,shape,dv,n)
    r1=R1_calculate(p,s,n)
    if l<=300:
        print('由于计算长度小于300km,可以不用考虑分布参数的影响，结果如下：')
        print(f'阻抗Z={complex(r1*l,x1*l)}，电纳Y={complex(0,b1*l)}')
        kr,kx,kb=modify_approximate(x1,b1,r1,l)
        z1=complex(r1,x1)
        y1=complex(0,b1)
        yy=np.sqrt(z1*y1)
        K1=np.sinh(yy*l)/(yy*l)
        K2=np.tanh(yy*l/2)/(yy*l/2)
        print('采用近似分布参数，结果如下:')
        print(f'阻抗Z={complex(r1*l*kr,x1*l*kx)}，电纳Y={complex(0,b1*l*kb)}')
        print('采用准确分布参数，结果如下L:')
        print(f'阻抗Z={K1*complex(r1*l,x1*l)}，电纳Y={K2*complex(0,b1*l*kb)}')
    if l>300:
        print('由于计算长度大于于300km,不能忽略分布参数的影响')
        kr,kx,kb=modify_approximate(x1,b1,r1,l)
        z1=complex(r1,x1)
        y1=complex(0,b1)
        yy=np.sqrt(z1*y1)
        K1=np.sinh(yy*l)/(yy*l)
        K2=np.tanh(yy*l/2)/(yy*l/2)
        print('采用近似分布参数，结果如下:')
        print(f'阻抗Z={complex(r1*l*kr,x1*l*kx)}，电纳Y={complex(0,b1*l*kb)}')
        print('采用准确分布参数，结果如下L:')
        print(f'阻抗Z={K1*complex(r1*l,x1*l)}，电纳Y={K2*complex(0,b1*l*kb)}')
l=eval(input('电线长度：(单位km)'))
p=eval(input('电阻率:'))
s=eval(input('单个导线的横截面积(mm2):'))
dh=eval(input('导线相间水平距离:(m)'))
r=eval(input('导线计算半径(m)'))
di=eval(input('导线每相内部间距(m)'))
n=eval(input('导线分裂数:'))
dv=eval(input('导线相间垂直距离(m),未提到就填写0'))
if n>1:
    shape=input('分裂导线间距：可选【正三角形，水平，正四边形，正五边形，正六边形】')
    main_judge(l,p,s,dh,r,di,shape,dv,n)
else: 
    main_judge(l,p,s,dh,r,di,dv,n)

由于计算长度大于于300km,不能忽略分布参数的影响
采用近似分布参数，结果如下:
阻抗Z=(11.90531151362403+140.339386064918j)，电纳Y=0.0019389687961904026j
采用准确分布参数，结果如下L:
阻抗Z=(11.930523420193273+140.43179808755627j)，电纳Y=(4.252594436097899e-06+0.0019852966303608526j)


# 进行封装

In [34]:
import tkinter as tk
from tkinter import ttk, messagebox
import numpy as np
import math

class TransmissionLineCalculator:
    def __init__(self, root):
        self.root = root
        self.root.title("输电线路参数计算器")
        self.root.geometry("1000x700")
        self.root.resizable(True, True)
        
        # 创建主框架
        main_frame = ttk.Frame(root, padding="10")
        main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        
        # 配置网格权重
        root.columnconfigure(0, weight=1)
        root.rowconfigure(0, weight=1)
        main_frame.columnconfigure(1, weight=1)
        
        # 创建输入区域
        self.create_input_section(main_frame)
        
        # 创建按钮区域
        self.create_button_section(main_frame)
        
        # 创建结果显示区域
        self.create_result_section(main_frame)
        
    def create_input_section(self, parent):
        # 输入区域标题
        input_label = ttk.Label(parent, text="线路参数输入", font=("Arial", 12, "bold"))
        input_label.grid(row=0, column=0, columnspan=2, pady=(0, 10), sticky=tk.W)
        
        # 电线长度
        ttk.Label(parent, text="电线长度 (km):").grid(row=1, column=0, sticky=tk.W, pady=5)
        self.length_entry = ttk.Entry(parent, width=20)
        self.length_entry.grid(row=1, column=1, sticky=(tk.W, tk.E), pady=5)
        
        # 电阻率
        ttk.Label(parent, text="电阻率 (Ω·mm²/km):").grid(row=2, column=0, sticky=tk.W, pady=5)
        self.resistivity_entry = ttk.Entry(parent, width=20)
        self.resistivity_entry.grid(row=2, column=1, sticky=(tk.W, tk.E), pady=5)
        
        # 导线横截面积
        ttk.Label(parent, text="单个导线横截面积 (mm²):").grid(row=3, column=0, sticky=tk.W, pady=5)
        self.area_entry = ttk.Entry(parent, width=20)
        self.area_entry.grid(row=3, column=1, sticky=(tk.W, tk.E), pady=5)
        
        # 导线相间水平距离
        ttk.Label(parent, text="导线相间水平距离(就是相线与相线的距离) (m):").grid(row=4, column=0, sticky=tk.W, pady=5)
        self.horizontal_dist_entry = ttk.Entry(parent, width=20)
        self.horizontal_dist_entry.grid(row=4, column=1, sticky=(tk.W, tk.E), pady=5)
        
        # 导线计算半径
        ttk.Label(parent, text="导线计算半径 (m):").grid(row=5, column=0, sticky=tk.W, pady=5)
        self.radius_entry = ttk.Entry(parent, width=20)
        self.radius_entry.grid(row=5, column=1, sticky=(tk.W, tk.E), pady=5)
        
        # 导线每相内部间距
        ttk.Label(parent, text="导线每相内部间距(分裂导线内部两两导线间距) (m):").grid(row=6, column=0, sticky=tk.W, pady=5)
        self.internal_dist_entry = ttk.Entry(parent, width=20)
        self.internal_dist_entry.grid(row=6, column=1, sticky=(tk.W, tk.E), pady=5)
        
        # 导线分裂数
        ttk.Label(parent, text="导线分裂数:").grid(row=7, column=0, sticky=tk.W, pady=5)
        self.split_number_var = tk.StringVar(value="1")
        split_number_frame = ttk.Frame(parent)
        split_number_frame.grid(row=7, column=1, sticky=(tk.W, tk.E), pady=5)
        for i in range(1, 7):
            ttk.Radiobutton(split_number_frame, text=str(i), variable=self.split_number_var, value=str(i)).pack(side=tk.LEFT)
        
        # 导线相间垂直距离
        ttk.Label(parent, text="导线相间垂直距离，没写就填0 (m):").grid(row=8, column=0, sticky=tk.W, pady=5)
        self.vertical_dist_entry = ttk.Entry(parent, width=20)
        self.vertical_dist_entry.grid(row=8, column=1, sticky=(tk.W, tk.E), pady=5)
        self.vertical_dist_entry.insert(0, "0")  # 默认值为0
        
        # 分裂导线形状
        ttk.Label(parent, text="分裂导线形状:").grid(row=9, column=0, sticky=tk.W, pady=5)
        self.shape_var = tk.StringVar(value="水平")
        shape_frame = ttk.Frame(parent)
        shape_frame.grid(row=9, column=1, sticky=(tk.W, tk.E), pady=5)
        shapes = ["水平(不分裂/2分裂)", "正三角形", "正四边形", "正五边形", "正六边形"]
        for shape in shapes:
            ttk.Radiobutton(shape_frame, text=shape, variable=self.shape_var, value=shape).pack(side=tk.LEFT)
    
    def create_button_section(self, parent):
        button_frame = ttk.Frame(parent)
        button_frame.grid(row=10, column=0, columnspan=2, pady=20)
        
        ttk.Button(button_frame, text="计算", command=self.calculate).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="清空", command=self.clear).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="退出", command=self.root.quit).pack(side=tk.LEFT, padx=5)
    
    def create_result_section(self, parent):
        # 结果显示区域标题
        result_label = ttk.Label(parent, text="计算结果", font=("Arial", 12, "bold"))
        result_label.grid(row=11, column=0, columnspan=2, pady=(10, 5), sticky=tk.W)
        
        # 创建文本框用于显示结果
        self.result_text = tk.Text(parent, width=70, height=15, wrap=tk.WORD)
        self.result_text.grid(row=12, column=0, columnspan=2, pady=5, sticky=(tk.W, tk.E, tk.N, tk.S))
        
        # 添加滚动条
        scrollbar = ttk.Scrollbar(parent, orient=tk.VERTICAL, command=self.result_text.yview)
        scrollbar.grid(row=12, column=2, sticky=(tk.N, tk.S))
        self.result_text.configure(yscrollcommand=scrollbar.set)
        
        # 配置父容器的行权重，使文本框可以扩展
        parent.rowconfigure(12, weight=1)
    
    def R1_calculate(self, p, s, n=1):
        """电阻率p,导线横截面积S，分裂数默认为1 (注意分裂导线S需要乘以n)"""
        # s单位是mm2, p单位是mm2*Ω/km
        return p / (s * n)
    
    def X_caculate(self, dh, r, di, shape="水平", dv=0, n=1):
        """dh:水平间距，di:相内间距, shape:形状, dv:垂直间距，r=计算半径，n：分裂数"""
        # shape可选：正三角形，水平，正四边形，正五边形，正六边形
        # dh与di，dv的单位要统一
        d1 = math.sqrt((dh/2)**2 + dv**2)
        if dv > 0:
            deq = ( dh * d1 * d1) ** (1/3)
        else:
            deq = (2 * dh * dh * dh) ** (1/3)
        
        alpha = 1
        if n == 2:
            alpha = 1
        elif n == 3 and shape == "水平":
            alpha = 1.26
        elif n == 3 and shape == "正三角形":
            alpha = 1
        elif n == 4:
            alpha = 1.12
        elif n == 5:
            alpha = 1.27
        elif n == 6:
            alpha = 1.4
        
        req = (r * (alpha * di) ** (n-1)) ** (1/n)
        
        x = 0.1445 * math.log10(deq / req) + 0.0157 / n
        b = 7.58e-6 / math.log10(deq / req)
        
        return x, b
    
    def modify_approximate(self, x1, b1, r1, l):
        kr = 1 - (1/3) * x1 * b1 * l * l
        kx = 1 - (1/6) * (x1 * b1 - r1 * r1 * b1 / x1) * l * l
        kb = 1 + (1/12) * x1 * b1 * l * l
        return kr, kx, kb
    
    def calculate(self):
        try:
            # 获取输入值
            l = float(self.length_entry.get())
            p = float(self.resistivity_entry.get())
            s = float(self.area_entry.get())
            dh = float(self.horizontal_dist_entry.get())
            r = float(self.radius_entry.get())
            di = float(self.internal_dist_entry.get())
            n = int(self.split_number_var.get())
            dv = float(self.vertical_dist_entry.get())
            shape = self.shape_var.get()
            
            # 验证输入
            if l < 0 or p < 0 or s < 0 or dh < 0 or r < 0 or di < 0 or n < 0 or dv < 0:
                messagebox.showerror("输入错误", "所有数值必须为正数（垂直距离可以为0）")
                return
            
            # 执行计算
            x1, b1 = self.X_caculate(dh, r, di, shape, dv, n)
            r1 = self.R1_calculate(p, s, n)
            
            # 清空结果文本框
            self.result_text.delete(1.0, tk.END)
            
            # 根据线路长度选择计算方法
            if l <= 300:
                self.result_text.insert(tk.END, f"由于计算长度 {l}km 小于300km，可以不用考虑分布参数的影响，结果如下：\n\n")
                
                # 集中参数模型
                z_concentrated = complex(r1 * l, x1 * l)
                y_concentrated = complex(0, b1 * l)
                self.result_text.insert(tk.END, f"集中参数模型：\n")
                self.result_text.insert(tk.END, f"阻抗 Z = {z_concentrated:.6f} Ω\n")
                self.result_text.insert(tk.END, f"电纳 Y = {y_concentrated:.6f} S\n\n")
                
                # 近似分布参数
                kr, kx, kb = self.modify_approximate(x1, b1, r1, l)
                z_approx = complex(r1 * l * kr, x1 * l * kx)
                y_approx = complex(0, b1 * l * kb)
                self.result_text.insert(tk.END, f"近似分布参数模型：\n")
                self.result_text.insert(tk.END, f"阻抗 Z = {z_approx:.6f} Ω\n")
                self.result_text.insert(tk.END, f"电纳 Y = {y_approx:.6f} S\n\n")
                
                # 准确分布参数
                z1 = complex(r1, x1)
                y1 = complex(0, b1)
                gamma = np.sqrt(z1 * y1)  # 传播常数
                K1 = np.sinh(gamma * l) / (gamma * l)
                K2 = np.tanh(gamma * l / 2) / (gamma * l / 2)
                z_exact = K1 * complex(r1 * l, x1 * l)
                y_exact = K2 * complex(0, b1 * l)
                
                self.result_text.insert(tk.END, f"准确分布参数模型：\n")
                self.result_text.insert(tk.END, f"阻抗 Z = {z_exact:.6f} Ω\n")
                self.result_text.insert(tk.END, f"电纳 Y = {y_exact:.6f} S\n")
                
            else:
                self.result_text.insert(tk.END, f"由于计算长度 {l}km 大于300km，不能忽略分布参数的影响\n\n")
                
                # 近似分布参数
                kr, kx, kb = self.modify_approximate(x1, b1, r1, l)
                z_approx = complex(r1 * l * kr, x1 * l * kx)
                y_approx = complex(0, b1 * l * kb)
                self.result_text.insert(tk.END, f"近似分布参数模型：\n")
                self.result_text.insert(tk.END, f"阻抗 Z = {z_approx:.6f} Ω\n")
                self.result_text.insert(tk.END, f"电纳 Y = {y_approx:.6f} S\n\n")
                
                # 准确分布参数
                z1 = complex(r1, x1)
                y1 = complex(0, b1)
                gamma = np.sqrt(z1 * y1)  # 传播常数
                K1 = np.sinh(gamma * l) / (gamma * l)
                K2 = np.tanh(gamma * l / 2) / (gamma * l / 2)
                z_exact = K1 * complex(r1 * l, x1 * l)
                y_exact = K2 * complex(0, b1 * l)
                
                self.result_text.insert(tk.END, f"准确分布参数模型：\n")
                self.result_text.insert(tk.END, f"阻抗 Z = {z_exact:.6f} Ω\n")
                self.result_text.insert(tk.END, f"电纳 Y = {y_exact:.6f} S\n")
                
        except ValueError as e:
            messagebox.showerror("输入错误", "请输入有效的数值")
        except Exception as e:
            messagebox.showerror("计算错误", f"计算过程中出现错误: {str(e)}")
    
    def clear(self):
        # 清空所有输入字段
        self.length_entry.delete(0, tk.END)
        self.resistivity_entry.delete(0, tk.END)
        self.area_entry.delete(0, tk.END)
        self.horizontal_dist_entry.delete(0, tk.END)
        self.radius_entry.delete(0, tk.END)
        self.internal_dist_entry.delete(0, tk.END)
        self.vertical_dist_entry.delete(0, tk.END)
        self.vertical_dist_entry.insert(0, "0")
        self.split_number_var.set("1")
        self.shape_var.set("水平")
        self.result_text.delete(1.0, tk.END)

if __name__ == "__main__":
    root = tk.Tk()
    app = TransmissionLineCalculator(root)
    root.mainloop()